<a href="https://www.kaggle.com/code/shehabahmed74/graduation-project-2-final-yolo-11-vs-yolo-8-fa?scriptVersionId=232807555" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Graduation Project 2: YOLOv8 vs. YOLOv11 Facial Recognition Demo

This notebook implements a Streamlit app to compare YOLOv8 and YOLOv11 for facial recognition. The project includes the following steps:
- **Step 1:** Install dependencies and verify GPU availability.
- **Step 2:** Fine-tune YOLOv11 on a facial recognition dataset.
- **Step 3:** Set up and deploy a Streamlit app to perform facial recognition and compare YOLOv8 and YOLOv11.
- **Step 4:** Test the app and verify the results.
- **Step 5:** Summarize the results and provide the public URL for the app.

The dataset used is `shehabahmed74/shehab-data-facial-recognition`, which contains three folders: `train`, `valid`, and `test`, each with `images` and `labels` subfolders. We will use `train` for training, `valid` for validation, and `test` for testing in the Streamlit app.

The notebook is structured to be professional and compatible with GitHub for project delivery.

**Execution Time Estimate:**
- With GPU: ~15–25 minutes (fine-tuning takes ~10–20 minutes for 10 epochs).
- Without GPU (on CPU): ~30–60 minutes (fine-tuning takes ~20–40 minutes for 10 epochs).
- To optimize performance, ensure a GPU runtime is selected (see Step 1.2).

---

## Step 1: Install Dependencies

This cell installs the required Python packages for the project, including Streamlit, ngrok, Ultralytics (for YOLOv11), and other dependencies.

**Instructions:**
- Run this cell to install the dependencies.
- If you encounter version conflicts, you may need to restart the runtime (`Runtime > Restart runtime`) and rerun this cell.

In [1]:
# Install dependencies
!pip install streamlit -q
!pip install pyngrok -q
!pip install ultralytics seaborn matplotlib pandas scipy kaggle pillow -q
print("Dependencies installed successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 56.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 94.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 17.5 MB/s eta 0:00:00a 0:00:01
Dependencies installed successfully!


In [2]:
# Upgrade pyngrok to the latest version using subprocess
import subprocess
import os

# Set the environment for the subprocess
env = os.environ.copy()
env["LC_ALL"] = "C.UTF-8"
env["LANG"] = "C.UTF-8"
env["LANGUAGE"] = "C.UTF-8"

print("Upgrading pyngrok...")
try:
    # Use subprocess to run the pip install command
    process = subprocess.run(
        ["pip", "install", "--upgrade", "pyngrok", "-q"],
        env=env,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    # Check if the command was successful
    if process.returncode == 0:
        print("pyngrok upgraded successfully!")
    else:
        print("Failed to upgrade pyngrok. Error output:")
        print(process.stderr)
        raise RuntimeError("pyngrok upgrade failed.")
except Exception as e:
    print(f"Error upgrading pyngrok: {str(e)}")
    raise

Upgrading pyngrok...
pyngrok upgraded successfully!


In [3]:
# Install dependencies
!pip install streamlit pyngrok ultralytics kaggle

## Step 1.1: Verify Installed Versions

This cell checks the installed versions of key packages to ensure compatibility.

**Instructions:**
- Run this cell to verify the versions of NumPy and Ultralytics.
- Expected versions (approximate):
  - NumPy: 2.1.x
  - Ultralytics: 8.3.x
- If there are compatibility issues, you may need to pin specific versions in the previous cell (e.g., `numpy==1.26.4 ultralytics==8.3.92`).

In [4]:
# Check installed versions
import numpy
import ultralytics
print(f"NumPy version: {numpy.__version__}")
print(f"Ultralytics version: {ultralytics.__version__}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
NumPy version: 1.26.4
Ultralytics version: 8.3.105


## Step 1.2: Verify GPU Availability

This cell checks if a GPU is available, as fine-tuning YOLOv11 is significantly faster on a GPU.

**Instructions:**
- Run this cell to verify GPU availability.
- If the output shows `CUDA available: True`, a GPU is available, and you can proceed.
- If the output shows `CUDA available: False`, switch to a GPU runtime:
  - Go to `Runtime > Change runtime type`.
  - Select `GPU` under "Hardware accelerator" (e.g., T4 GPU).
  - Click `Save` and restart the runtime (`Runtime > Restart runtime`).
  - Rerun all cells above this one.
- **Note:** If a GPU is not available, the notebook will fall back to CPU, but fine-tuning will be slower (~20–40 minutes for 10 epochs vs. ~10–20 minutes on a GPU).

In [5]:
# Verify GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Number of GPUs: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("WARNING: No GPU detected. Fine-tuning will be slower on CPU. See instructions above to switch to a GPU runtime.")

CUDA available: True
Number of GPUs: 1
GPU Name: Tesla P100-PCIE-16GB


## Step 1.3: Set Up ngrok Authtoken

This cell sets up ngrok to create a public URL for the Streamlit app.

**Instructions:**
- The ngrok authtoken is already provided.
- Run this cell to set up ngrok.
- If you encounter issues with the authtoken, ensure it is valid by checking your ngrok dashboard at [ngrok.com](https://ngrok.com).

In [6]:
# Set up ngrok authtoken
from pyngrok import ngrok

# Set the ngrok authtoken
ngrok.set_auth_token("2ufQbxOVW7rhcqlvOyIFlEE6Y8l_7rFynmFyQYqUF2bioBaW3")

print("ngrok authtoken set up successfully!")

ngrok authtoken set up successfully!                                                                


## Step 2: Fine-Tune YOLOv11 for Facial Recognition

This step fine-tunes the YOLOv11 nano model (`yolo11n.pt`) on the facial recognition dataset (`shehabahmed74/shehab-data-facial-recognition`). The fine-tuned model will be saved as `yolo11n_finetuned.pt` and used in the Streamlit app for more specific facial recognition labels (e.g., "face" instead of "person").

The dataset contains three folders: `train`, `valid`, and `test`. We will use:
- `train` for training.
- `valid` for validation.
- `test` for testing in the Streamlit app.

The dataset paths, number of classes (`nc`), and class names (`names`) are pre-configured based on prior testing. We assume:
- There is one class (`face`), so `nc: 1` and `names: ['face']`.

**This cell will:**
- Download the dataset (`shehabahmed74/shehab-data-facial-recognition`) using `kagglehub`.
- Unzip the dataset and verify its structure.
- Dynamically set the `train` and `val` paths in `data.yaml` based on the actual dataset structure.
- Fine-tune the YOLOv11 model.

**Instructions:**
- Run the next cell (Cell 10) to download the dataset and fine-tune the model.
- This step takes the longest:
  - With GPU: ~10–20 minutes for 10 epochs.
  - Without GPU (on CPU): ~20–40 minutes for 10 epochs.
- If you encounter errors (e.g., missing dataset paths), the dataset structure may differ from the expected layout. The cell will print the dataset structure to help you debug. You can also manually check the structure by adding a new code cell with:
  ```python
  !ls -R /root/.cache/kagglehub/datasets/shehabahmed74/shehab-data-facial-recognition/versions/1

In [7]:
# # Step 2: Fine-tune YOLOv11
# from ultralytics import YOLO
# import os
# import torch
# import kagglehub
# import zipfile
# import glob
# import shutil

# # Function to print directory structure using Python
# def print_directory_structure(directory, indent=0, max_files=5):
#     """Recursively print the directory structure with a limit on the number of files displayed per directory."""
#     try:
#         print("  " * indent + f"Directory: {directory}")
#         items = sorted(os.listdir(directory))
#         for item in items:
#             item_path = os.path.join(directory, item)
#             if os.path.isdir(item_path):
#                 print("  " * (indent + 1) + f"Subdirectory: {item}")
#                 print_directory_structure(item_path, indent + 2, max_files)
#             else:
#                 if max_files > 0:
#                     print("  " * (indent + 1) + f"File: {item}")
#                     max_files -= 1
#                 elif max_files == 0:
#                     print("  " * (indent + 1) + "... (additional files omitted)")
#                     break
#     except Exception as e:
#         print("  " * (indent + 1) + f"Error accessing directory: {str(e)}")

# # Function to download and process dataset
# def download_and_process_dataset():
#     print("Downloading and Processing Dataset...")
#     try:
#         # Download dataset
#         dataset_path = kagglehub.dataset_download("shehabahmed74/shehab-data-facial-recognition")
#         print(f"Dataset downloaded to: {dataset_path}")

#         # Process dataset (unzip if necessary)
#         if os.path.isdir(dataset_path):
#             processed_path = dataset_path
#         elif os.path.isfile(dataset_path) and dataset_path.endswith('.zip'):
#             unzip_dir = os.path.join(os.path.dirname(dataset_path), "unzipped_dataset")
#             os.makedirs(unzip_dir, exist_ok=True)
#             with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
#                 zip_ref.extractall(unzip_dir)
#             processed_path = unzip_dir
#         else:
#             raise ValueError("Dataset is neither a recognized zip file nor a usable directory.")

#         print("Dataset processing completed!")
#         return processed_path
#     except Exception as e:
#         print(f"Error downloading or processing dataset: {str(e)}")
#         raise

# # Download and process the dataset
# dataset_path = download_and_process_dataset()

# # Verify the dataset structure using Python
# print("\nDataset structure:")
# print_directory_structure(dataset_path)

# # Search for image files in the dataset
# print("\nSearching for image files in the dataset...")
# image_extensions = ["*.jpg", "*.jpeg", "*.png"]
# image_files = []
# for ext in image_extensions:
#     image_files.extend(glob.glob(os.path.join(dataset_path, "**", ext), recursive=True))

# if not image_files:
#     raise FileNotFoundError("No image files found in the dataset. Check the dataset structure above.")
# else:
#     print(f"Found {len(image_files)} image files. Examples:")
#     for img in image_files[:5]:  # Print first 5 image paths
#         print(img)

# # Search for label files in the dataset
# print("\nSearching for label files in the dataset...")
# label_files = glob.glob(os.path.join(dataset_path, "**", "*.txt"), recursive=True)
# if not label_files:
#     print("Warning: No label files found in the dataset. Check the dataset structure above.")
# else:
#     print(f"Found {len(label_files)} label files. Examples:")
#     for lbl in label_files[:5]:  # Print first 5 label paths
#         print(lbl)

# # Create a dictionary mapping image filenames (without extension) to their paths
# image_dict = {os.path.splitext(os.path.basename(img))[0]: img for img in image_files}
# label_dict = {os.path.splitext(os.path.basename(lbl))[0]: lbl for lbl in label_files}

# # Find images that have corresponding labels
# paired_files = []
# for img_name in image_dict.keys():
#     if img_name in label_dict:
#         paired_files.append((image_dict[img_name], label_dict[img_name]))

# if not paired_files:
#     raise ValueError("No images have corresponding label files. Check the dataset structure.")

# print(f"\nFound {len(paired_files)} image-label pairs.")

# # Split the paired files into training and validation sets (80% train, 20% val)
# paired_files.sort()  # Sort to ensure consistent splitting
# split_idx = int(0.8 * len(paired_files))
# train_pairs = paired_files[:split_idx]
# val_pairs = paired_files[split_idx:]

# # Create directories for training and validation
# train_images_dir = os.path.join(dataset_path, "custom_train", "images")
# train_labels_dir = os.path.join(dataset_path, "custom_train", "labels")
# val_images_dir = os.path.join(dataset_path, "custom_valid", "images")
# val_labels_dir = os.path.join(dataset_path, "custom_valid", "labels")

# # Remove existing directories to avoid conflicts
# print("\nRemoving existing custom_train and custom_valid directories (if any)...")
# for dir_path in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir]:
#     if os.path.exists(dir_path):
#         shutil.rmtree(dir_path)

# # Recreate the directories
# os.makedirs(train_images_dir, exist_ok=True)
# os.makedirs(train_labels_dir, exist_ok=True)
# os.makedirs(val_images_dir, exist_ok=True)
# os.makedirs(val_labels_dir, exist_ok=True)

# # Copy images and labels to the respective directories (symbolic links to save space)
# print("\nOrganizing images and labels into train and valid directories...")
# for img_path, lbl_path in train_pairs:
#     img_name = os.path.basename(img_path)
#     lbl_name = os.path.basename(lbl_path)
#     img_symlink = os.path.join(train_images_dir, img_name)
#     lbl_symlink = os.path.join(train_labels_dir, lbl_name)
#     # Create symbolic links, overwriting if they exist
#     if os.path.exists(img_symlink):
#         os.remove(img_symlink)
#     if os.path.exists(lbl_symlink):
#         os.remove(lbl_symlink)
#     os.symlink(img_path, img_symlink)
#     os.symlink(lbl_path, lbl_symlink)

# for img_path, lbl_path in val_pairs:
#     img_name = os.path.basename(img_path)
#     lbl_name = os.path.basename(lbl_path)
#     img_symlink = os.path.join(val_images_dir, img_name)
#     lbl_symlink = os.path.join(val_labels_dir, lbl_name)
#     # Create symbolic links, overwriting if they exist
#     if os.path.exists(img_symlink):
#         os.remove(img_symlink)
#     if os.path.exists(lbl_symlink):
#         os.remove(lbl_symlink)
#     os.symlink(img_path, img_symlink)
#     os.symlink(lbl_path, lbl_symlink)

# # Define the paths for data.yaml
# train_images_path = train_images_dir
# val_images_path = val_images_dir

# # Verify the new directories
# print(f"\nNew training images path: {train_images_path}")
# print(f"Number of training images: {len(glob.glob(os.path.join(train_images_path, '*')))}")
# print(f"New training labels path: {train_labels_dir}")
# print(f"Number of training labels: {len(glob.glob(os.path.join(train_labels_dir, '*')))}")
# print(f"New validation images path: {val_images_path}")
# print(f"Number of validation images: {len(glob.glob(os.path.join(val_images_path, '*')))}")
# print(f"New validation labels path: {val_labels_dir}")
# print(f"Number of validation labels: {len(glob.glob(os.path.join(val_labels_dir, '*')))}")

# # Check if the paths exist and are not empty
# if not os.path.exists(train_images_path) or not glob.glob(os.path.join(train_images_path, '*')):
#     raise FileNotFoundError(f"Training images path is empty or not found: {train_images_path}")
# if not os.path.exists(val_images_path) or not glob.glob(os.path.join(val_images_path, '*')):
#     raise FileNotFoundError(f"Validation images path is empty or not found: {val_images_path}")
# if not os.path.exists(train_labels_dir) or not glob.glob(os.path.join(train_labels_dir, '*')):
#     raise FileNotFoundError(f"Training labels path is empty or not found: {train_labels_dir}")
# if not os.path.exists(val_labels_dir) or not glob.glob(os.path.join(val_labels_dir, '*')):
#     raise FileNotFoundError(f"Validation labels path is empty or not found: {val_labels_dir}")

# # Load the pre-trained YOLOv11 nano model
# model = YOLO("yolo11n.pt")

# # Define the data.yaml with the correct paths
# data_yaml = f"""
# train: {train_images_path}
# val: {val_images_path}
# nc: 1  # Number of classes (assumed to be 1 for 'face')
# names: ['face']  # Class names (assumed to be 'face')
# """

# # Save the data.yaml file
# with open("data.yaml", "w") as f:
#     f.write(data_yaml)

# # Determine the device (GPU if available, else CPU)
# device = 0 if torch.cuda.is_available() else 'cpu'
# print(f"Using device: {'GPU' if device == 0 else 'CPU'}")

# # Fine-tune the model
# try:
#     model.train(
#         data="data.yaml",
#         epochs=10,  # Use fewer epochs for a quick test; increase for better results
#         imgsz=640,
#         batch=16,
#         device=device  # Use GPU if available, else CPU
#     )
# except Exception as e:
#     print(f"Error during fine-tuning: {str(e)}")
#     raise

# # Save the fine-tuned model
# model.save("yolo11n_finetuned.pt")
# print("Fine-tuning complete! Model saved as 'yolo11n_finetuned.pt'")

In [8]:
### Cell: Step 2
import os
import shutil
import random

# Step 1: Copy dataset to writable directory
source_dataset_dir = "/kaggle/input/shehab-data-facial-recognition"
working_dataset_dir = "/kaggle/working/shehab-data-facial-recognition"
if os.path.exists(working_dataset_dir):
    shutil.rmtree(working_dataset_dir)
shutil.copytree(source_dataset_dir, working_dataset_dir)
print("Step 1: Dataset copied to writable directory")

# Step 2: Define directory paths
train_images_dir = os.path.join(working_dataset_dir, "custom_train", "images")
train_labels_dir = os.path.join(working_dataset_dir, "custom_train", "labels")
val_images_dir = os.path.join(working_dataset_dir, "custom_val", "images")
val_labels_dir = os.path.join(working_dataset_dir, "custom_val", "labels")
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)
print("Step 2: Directory paths defined")

# Step 3: Find images and labels
image_extensions = (".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG")
all_images = []
all_labels = []
for root, dirs, files in os.walk(working_dataset_dir):
    if "custom_train" in root or "custom_val" in root:
        continue
    for file in files:
        if file.lower().endswith(image_extensions):
            all_images.append(os.path.join(root, file))
        elif file.endswith(".txt"):
            all_labels.append(os.path.join(root, file))
print(f"Step 3: Found {len(all_images)} images, {len(all_labels)} labels")

# Step 4: Pair images with labels and deduplicate
image_label_pairs = []
seen_names = set()
for image_path in all_images:
    image_name = os.path.splitext(os.path.basename(image_path))[0]
    if image_name in seen_names:
        continue  # Skip duplicates
    for label_path in all_labels:
        label_name = os.path.splitext(os.path.basename(label_path))[0]
        if image_name == label_name and os.path.exists(image_path) and os.path.exists(label_path):
            image_label_pairs.append((image_path, label_path))
            seen_names.add(image_name)
            break
print(f"Step 4: Found {len(image_label_pairs)} image-label pairs after deduplication")

# Step 5: Split into train and validation sets
random.shuffle(image_label_pairs)
split_idx = int(0.8 * len(image_label_pairs))
train_pairs = image_label_pairs[:split_idx]
val_pairs = image_label_pairs[split_idx:]
print(f"Step 5: Split into {len(train_pairs)} training pairs, {len(val_pairs)} validation pairs")

# Step 6: Move files to training directories
train_images_moved = 0
train_labels_moved = 0
train_skipped = 0
for image_path, label_path in train_pairs:
    if os.path.exists(image_path) and os.path.exists(label_path):
        shutil.move(image_path, os.path.join(train_images_dir, os.path.basename(image_path)))
        shutil.move(label_path, os.path.join(train_labels_dir, os.path.basename(label_path)))
        train_images_moved += 1
        train_labels_moved += 1
    else:
        train_skipped += 1
print(f"Step 6: Moved {train_images_moved} images, {train_labels_moved} labels to training directories, skipped {train_skipped} pairs")

# Step 7: Move files to validation directories
val_images_moved = 0
val_labels_moved = 0
val_skipped = 0
for image_path, label_path in val_pairs:
    if os.path.exists(image_path) and os.path.exists(label_path):
        shutil.move(image_path, os.path.join(val_images_dir, os.path.basename(image_path)))
        shutil.move(label_path, os.path.join(val_labels_dir, os.path.basename(label_path)))
        val_images_moved += 1
        val_labels_moved += 1
    else:
        val_skipped += 1
print(f"Step 7: Moved {val_images_moved} images, {val_labels_moved} labels to validation directories, skipped {val_skipped} pairs")

# Step 8: Verify pairing
train_images = os.listdir(train_images_dir)
train_labels = os.listdir(train_labels_dir)
val_images = os.listdir(val_images_dir)
val_labels = os.listdir(val_labels_dir)
train_pairs_verified = sum(1 for img in train_images if f"{os.path.splitext(img)[0]}.txt" in train_labels)
val_pairs_verified = sum(1 for img in val_images if f"{os.path.splitext(img)[0]}.txt" in val_labels)
print(f"Step 8: Verified {train_pairs_verified} pairs in custom_train, {val_pairs_verified} pairs in custom_val")

Step 1: Dataset copied to writable directory
Step 2: Directory paths defined
Step 3: Found 25262 images, 25262 labels
Step 4: Found 23129 image-label pairs after deduplication
Step 5: Split into 18503 training pairs, 4626 validation pairs
Step 6: Moved 18503 images, 18503 labels to training directories, skipped 0 pairs
Step 7: Moved 4626 images, 4626 labels to validation directories, skipped 0 pairs
Step 8: Verified 18503 pairs in custom_train, 4626 pairs in custom_val


In [9]:
# # Step 3: Create the Streamlit app
# import os

# # Define the Streamlit app code
# streamlit_app_code = """
# import streamlit as st
# import os
# import glob
# import random
# from PIL import Image
# from ultralytics import YOLO

# # Load the fine-tuned YOLOv11 model
# @st.cache_resource
# def load_model():
#     return YOLO("yolo11n_finetuned.pt")

# model = load_model()

# # Streamlit app
# st.title("YOLOv11 Facial Recognition Demo")

# # Step 1: Dataset Path Verification
# st.header("Step 1: Dataset Path Verification")
# dataset_path = st.text_input("Enter the dataset path from Step 2:", value="/root/.cache/kagglehub/datasets/shehabahmed74/shehab-data-facial-recognition/versions/1")
# if dataset_path:
#     # Check for 'test' directory
#     test_path = os.path.join(dataset_path, "test")
#     if not os.path.exists(test_path):
#         st.error(f"'test' directory not found: {test_path}. Ensure Step 2 completed successfully and enter the correct path.")
#         test_images = []
#     else:
#         # Initialize test images list
#         test_images = []
#         # Check for nested 'test/test/images' directory (where images are located)
#         test_images_path = os.path.join(test_path, "test", "images")
#         st.write(f"Checking for images in: {test_images_path}")
#         if os.path.exists(test_images_path):
#             # Look for images with any case variation of .jpg, .png, .jpeg
#             test_images = glob.glob(os.path.join(test_images_path, "*.[jJ][pP][gG]")) + \
#                           glob.glob(os.path.join(test_images_path, "*.[pP][nN][gG]")) + \
#                           glob.glob(os.path.join(test_images_path, "*.[jJ][pP][eE][gG]"))
#             st.write(f"Found {len(test_images)} images in 'test/test/images'")
#             if test_images:
#                 st.success(f"Test images directory found: {test_images_path}")
#                 st.write(f"Found {len(test_images)} test images.")
#                 st.write(f"Sample images: {test_images[:5]}")
#                 # Verify that the first image is readable
#                 try:
#                     with open(test_images[0], "rb") as f:
#                         f.read(1)
#                     st.write(f"Successfully read first image: {test_images[0]}")
#                 except Exception as e:
#                     st.error(f"Error reading image {test_images[0]}: {str(e)}")
#                     test_images = []
#             else:
#                 st.warning(f"Test images directory exists but no .jpg, .png, or .jpeg images found in: {test_images_path}")
#                 st.write(f"Files in 'test/test/images': {os.listdir(test_images_path)[:5]}")
#         else:
#             st.warning(f"'test/test/images' directory not found: {test_images_path}")
#             st.write(f"Contents of 'test': {os.listdir(test_path)[:5]}")
#             # Fallback: Check for images directly in 'test/test'
#             test_images_path = os.path.join(test_path, "test")
#             st.write(f"Checking for images directly in: {test_images_path}")
#             if os.path.exists(test_images_path):
#                 test_images = glob.glob(os.path.join(test_images_path, "*.[jJ][pP][gG]")) + \
#                               glob.glob(os.path.join(test_images_path, "*.[pP][nN][gG]")) + \
#                               glob.glob(os.path.join(test_images_path, "*.[jJ][pP][eE][gG]"))
#                 st.write(f"Found {len(test_images)} images directly in 'test/test'")
#                 if test_images:
#                     st.success(f"Test images found in nested 'test/test' directory: {test_images_path}")
#                     st.write(f"Found {len(test_images)} test images.")
#                     st.write(f"Sample images: {test_images[:5]}")
#                 else:
#                     st.warning(f"No .jpg, .png, or .jpeg images found in {test_images_path}")
#                     st.write(f"Files in 'test/test': {os.listdir(test_images_path)[:5]}")
#             else:
#                 st.warning(f"'test/test' directory not found: {test_images_path}")

#         # Final check: If no images are found, notify the user
#         if not test_images:
#             st.info("No test images found. You can still upload an external image for inference below.")

# # Step 2: Test YOLOv11 Facial Recognition
# st.header("Step 2: Test YOLOv11 Facial Recognition")

# # Option to select image source
# image_source_options = ["Use a random image from the test dataset", "Upload an external image"]
# image_source = st.radio(
#     "Select image source for facial recognition:",
#     image_source_options,
#     index=0 if test_images else 1  # Default to random if test images exist, otherwise upload
# )

# # Disable the random image option if no test images are found
# if not test_images and image_source == "Use a random image from the test dataset":
#     st.warning("No test images found. Please select 'Upload an external image' instead.")
#     image_source = "Upload an external image"

# # Initialize image_path
# image_path = None

# # Handle random image selection
# if image_source == "Use a random image from the test dataset" and test_images:
#     image_path = random.choice(test_images)
#     st.write(f"Selected random image: {os.path.basename(image_path)}")

# # Always show the file uploader for external images
# if image_source == "Upload an external image":
#     uploaded_file = st.file_uploader("Upload an external image for facial recognition", type=["jpg", "jpeg", "png"])
#     if uploaded_file is not None:
#         # Save the uploaded file temporarily
#         image_path = os.path.join("temp", uploaded_file.name)
#         st.write(f"Saving uploaded file to: {image_path}")
#         try:
#             os.makedirs("temp", exist_ok=True)
#             with open(image_path, "wb") as f:
#                 f.write(uploaded_file.getbuffer())
#             st.write(f"Uploaded image: {uploaded_file.name}")
#             # Verify the file was saved
#             if os.path.exists(image_path):
#                 st.write(f"File successfully saved at: {image_path}")
#             else:
#                 st.error(f"Failed to save the uploaded file at: {image_path}")
#                 image_path = None
#         except Exception as e:
#             st.error(f"Error saving uploaded file: {str(e)}")
#             image_path = None
#     else:
#         st.info("Please upload an image to proceed.")

# # Run inference if an image is selected
# if st.button("Run Inference"):
#     if image_path:
#         # Load and display the image
#         try:
#             image = Image.open(image_path)
#             st.image(image, caption="Input Image", use_column_width=True)
#         except Exception as e:
#             st.error(f"Error loading image {image_path}: {str(e)}")
#             image_path = None

#         if image_path:
#             # Run YOLOv11 inference
#             with st.spinner("Running facial recognition..."):
#                 try:
#                     results = model(image)
#                     # Display the results
#                     st.header("Step 3: Results")
#                     # Plot the results
#                     annotated_image = results[0].plot()  # Get the annotated image with bounding boxes
#                     st.image(annotated_image, caption="Detected Faces", use_column_width=True)

#                     # Display detection details
#                     detections = results[0].boxes
#                     st.write(f"Number of faces detected: {len(detections)}")
#                     for i, det in enumerate(detections):
#                         conf = det.conf.item()
#                         bbox = det.xyxy[0].tolist()
#                         st.write(f"Face {i+1}: Confidence = {conf:.2f}, Bounding Box = {bbox}")
#                 except Exception as e:
#                     st.error(f"Error during inference: {str(e)}")
#     else:
#         st.error("Please select a random test image or upload an external image to run inference.")

# # Step 4: Summary and Conclusion
# st.header("Step 4: Summary and Conclusion")
# st.write("This demo showcases the YOLOv11 model fine-tuned for facial recognition.")
# st.write("The model was fine-tuned on the dataset provided in Step 2 and tested on either a random test image or an uploaded external image.")
# st.write("For detailed performance metrics and comparisons, refer to the notebook (Step 3.1).")
# """

# # Write the Streamlit app code to app.py
# with open("app.py", "w") as f:
#     f.write(streamlit_app_code)

# # Verify that app.py was created
# if os.path.exists("app.py"):
#     print("Streamlit app created successfully at app.py!")
# else:
#     raise FileNotFoundError("Failed to create app.py")

### Step 3: Fine-tune YOLOv11 model

In [10]:
# Step 0: Install required libraries
!pip install ultralytics
print("Ultralytics installed successfully")

Ultralytics installed successfully


In [11]:
# Step 0: Install required libraries
!pip install ultralytics
print("Ultralytics installed successfully")

Ultralytics installed successfully


To fix warning message which appeared when run cell 3

In [12]:
# Step 10.5: Fix label files to ensure class ID is 0
import os

# Define directories
train_labels_dir = "/kaggle/working/shehab-data-facial-recognition/custom_train/labels"
val_labels_dir = "/kaggle/working/shehab-data-facial-recognition/custom_val/labels"

# Function to fix a single label file
def fix_label_file(label_path):
    try:
        with open(label_path, "r") as f:
            lines = f.readlines()
        # Fix class ID to 0
        fixed_lines = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 5:  # Ensure the line has at least 5 parts (class_id, x, y, w, h)
                parts[0] = "0"  # Set class ID to 0
                fixed_lines.append(" ".join(parts) + "\n")
        # Write the fixed lines back to the file
        with open(label_path, "w") as f:
            f.writelines(fixed_lines)
    except Exception as e:
        print(f"Error fixing {label_path}: {str(e)}")

# Fix training labels
train_fixed = 0
for label_file in os.listdir(train_labels_dir):
    if label_file.endswith(".txt"):
        fix_label_file(os.path.join(train_labels_dir, label_file))
        train_fixed += 1

# Fix validation labels
val_fixed = 0
for label_file in os.listdir(val_labels_dir):
    if label_file.endswith(".txt"):
        fix_label_file(os.path.join(val_labels_dir, label_file))
        val_fixed += 1

print(f"Fixed {train_fixed} training labels and {val_fixed} validation labels")

Fixed 18503 training labels and 4626 validation labels


In [13]:
# Step 10: Fine-tune YOLOv11 model
from ultralytics import YOLO
import os

# Create data.yaml if it doesn't exist
data_yaml_path = "/kaggle/working/shehab-data-facial-recognition/data.yaml"
if not os.path.exists(data_yaml_path):
    data_yaml = """
    train: /kaggle/working/shehab-data-facial-recognition/custom_train/images
    val: /kaggle/working/shehab-data-facial-recognition/custom_val/images
    nc: 1
    names: ['face']
    """
    with open(data_yaml_path, "w") as f:
        f.write(data_yaml)
    print("Created data.yaml")

# Load the pre-trained YOLOv11 model
model = YOLO("yolo11n.pt")  # This will download the model if not present

# Fine-tune the model on your dataset
model.train(
    data=data_yaml_path,
    epochs=10,
    imgsz=640,
    batch=8,
    device=0,
    project="/kaggle/working/runs/train",
    name="yolo11n_finetune"
)

# Save the fine-tuned model
model.save("/kaggle/working/yolo11n_finetuned.pt")
print("Fine-tuning complete. Model saved as 'yolo11n_finetuned.pt'")

Created data.yaml
Ultralytics 8.3.105 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/kaggle/working/shehab-data-facial-recognition/data.yaml, epochs=10, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=/kaggle/working/runs/train, name=yolo11n_finetune6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_c

100%|██████████| 755k/755k [00:00<00:00, 17.3MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /kaggle/working/shehab-data-facial-recognition/custom_train/labels... 18503 images, 0 backgrounds, 0 corrupt: 100%|██████████| 18503/18503 [00:15<00:00, 1214.22it/s]


train: New cache created: /kaggle/working/shehab-data-facial-recognition/custom_train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...


10 epochs completed in 0.940 hours.
Optimizer stripped from /kaggle/working/runs/train/yolo11n_finetune6/weights/last.pt, 5.5MB
Optimizer stripped from /kaggle/working/runs/train/yolo11n_finetune6/weights/best.pt, 5.5MB

Validating /kaggle/working/runs/train/yolo11n_finetune6/weights/best.pt...
Ultralytics 8.3.105 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 290/290 [00:28<00:00, 10.24it/s]


                   all       4626       4626          1          1      0.995      0.995


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.2ms preprocess, 2.1ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /kaggle/working/runs/train/yolo11n_finetune6
Fine-tuning complete. Model saved as 'yolo11n_finetuned.pt'


# Step 4: Create the Streamlit app

In [14]:
# Step 4: Create the Streamlit app
import os

# Check if the fine-tuned model file exists
model_file = "yolo11n_finetuned.pt"
if not os.path.exists(model_file):
    raise FileNotFoundError(f"The fine-tuned model file '{model_file}' does not exist. Please run the fine-tuning step (Cell 10) to create it.")

# Define the Streamlit app code
streamlit_app_code = """
import streamlit as st
import os
import glob
import random
from PIL import Image
from ultralytics import YOLO

# Load the fine-tuned YOLOv11 model
@st.cache_resource
def load_model():
    return YOLO("yolo11n_finetuned.pt")

model = load_model()

# Initialize session state for image_path
if 'image_path' not in st.session_state:
    st.session_state.image_path = None

# Streamlit app
st.title("YOLOv11 Facial Recognition Demo")

# Step 1: Dataset Path Verification
st.header("Step 1: Dataset Path Verification")
dataset_path = st.text_input("Enter the dataset path from Step 2:", value="/root/.cache/kagglehub/datasets/shehabahmed74/shehab-data-facial-recognition/versions/1")
if dataset_path:
    # Check for 'test' directory
    test_path = os.path.join(dataset_path, "test")
    if not os.path.exists(test_path):
        st.error(f"'test' directory not found: {test_path}. Ensure Step 2 completed successfully and enter the correct path.")
        test_images = []
    else:
        # Initialize test images list
        test_images = []
        # Check for nested 'test/test/images' directory (where images are located)
        test_images_path = os.path.join(test_path, "test", "images")
        st.write(f"Checking for images in: {test_images_path}")
        if os.path.exists(test_images_path):
            # Look for images with any case variation of .jpg, .png, .jpeg
            test_images = glob.glob(os.path.join(test_images_path, "*.[jJ][pP][gG]")) + \
                          glob.glob(os.path.join(test_images_path, "*.[pP][nN][gG]")) + \
                          glob.glob(os.path.join(test_images_path, "*.[jJ][pP][eE][gG]"))
            st.write(f"Found {len(test_images)} images in 'test/test/images'")
            if test_images:
                st.success(f"Test images directory found: {test_images_path}")
                st.write(f"Found {len(test_images)} test images.")
                st.write(f"Sample images: {test_images[:5]}")
                # Verify that the first image is readable
                try:
                    with open(test_images[0], "rb") as f:
                        f.read(1)
                    st.write(f"Successfully read first image: {test_images[0]}")
                except Exception as e:
                    st.error(f"Error reading image {test_images[0]}: {str(e)}")
                    test_images = []
            else:
                st.warning(f"Test images directory exists but no .jpg, .png, or .jpeg images found in: {test_images_path}")
                st.write(f"Files in 'test/test/images': {os.listdir(test_images_path)[:5]}")
        else:
            st.warning(f"'test/test/images' directory not found: {test_images_path}")
            st.write(f"Contents of 'test': {os.listdir(test_path)[:5]}")
            # Fallback: Check for images directly in 'test/test'
            test_images_path = os.path.join(test_path, "test")
            st.write(f"Checking for images directly in: {test_images_path}")
            if os.path.exists(test_images_path):
                test_images = glob.glob(os.path.join(test_images_path, "*.[jJ][pP][gG]")) + \
                              glob.glob(os.path.join(test_images_path, "*.[pP][nN][gG]")) + \
                              glob.glob(os.path.join(test_images_path, "*.[jJ][pP][eE][gG]"))
                st.write(f"Found {len(test_images)} images directly in 'test/test'")
                if test_images:
                    st.success(f"Test images found in nested 'test/test' directory: {test_images_path}")
                    st.write(f"Found {len(test_images)} test images.")
                    st.write(f"Sample images: {test_images[:5]}")
                else:
                    st.warning(f"No .jpg, .png, or .jpeg images found in {test_images_path}")
                    st.write(f"Files in 'test/test': {os.listdir(test_images_path)[:5]}")
            else:
                st.warning(f"'test/test' directory not found: {test_images_path}")

        # Final check: If no images are found, notify the user
        if not test_images:
            st.info("No test images found. You can still upload an external image for inference below.")

# Step 2: Test YOLOv11 Facial Recognition
st.header("Step 2: Test YOLOv11 Facial Recognition")

# Option to select image source
image_source_options = ["Use a random image from the test dataset", "Upload an external image"]
image_source = st.radio(
    "Select image source for facial recognition:",
    image_source_options,
    index=0 if test_images else 1  # Default to random if test images exist, otherwise upload
)

# Disable the random image option if no test images are found
if not test_images and image_source == "Use a random image from the test dataset":
    st.warning("No test images found. Please select 'Upload an external image' instead.")
    image_source = "Upload an external image"

# Initialize local image_path for this run
image_path = None

# Handle random image selection
if image_source == "Use a random image from the test dataset" and test_images:
    image_path = random.choice(test_images)
    st.write(f"Selected random image: {os.path.basename(image_path)}")
    st.session_state.image_path = image_path

# Handle external image upload
if image_source == "Upload an external image":
    uploaded_file = st.file_uploader("Upload an external image for facial recognition", type=["jpg", "jpeg", "png"])
    if uploaded_file is not None:
        # Save the uploaded file temporarily
        image_path = os.path.join("temp", uploaded_file.name)
        st.write(f"Saving uploaded file to: {image_path}")
        try:
            os.makedirs("temp", exist_ok=True)
            with open(image_path, "wb") as f:
                f.write(uploaded_file.getbuffer())
            st.write(f"Uploaded image: {uploaded_file.name}")
            # Verify the file was saved
            if os.path.exists(image_path):
                st.write(f"File successfully saved at: {image_path}")
                st.session_state.image_path = image_path  # Store in session state
            else:
                st.error(f"Failed to save the uploaded file at: {image_path}")
                st.session_state.image_path = None
        except Exception as e:
            st.error(f"Error saving uploaded file: {str(e)}")
            st.session_state.image_path = None
    else:
        st.info("Please upload an image to proceed.")

    # Display current uploaded image status
    if st.session_state.image_path and image_source == "Upload an external image":
        st.success(f"Image ready for inference: {os.path.basename(st.session_state.image_path)}")
        if st.button("Clear Uploaded Image"):
            st.session_state.image_path = None
            st.experimental_rerun()

# Run inference if an image is selected
if st.button("Run Inference"):
    # Use the session state image_path if available, otherwise use the local image_path
    current_image_path = st.session_state.image_path if st.session_state.image_path else image_path
    if current_image_path:
        # Load and display the image
        try:
            image = Image.open(current_image_path)
            st.image(image, caption="Input Image", use_column_width=True)
        except Exception as e:
            st.error(f"Error loading image {current_image_path}: {str(e)}")
            st.session_state.image_path = None
            current_image_path = None

        if current_image_path:
            # Run YOLOv11 inference
            with st.spinner("Running facial recognition..."):
                try:
                    results = model(image)
                    # Display the results
                    st.header("Step 3: Results")
                    # Plot the results
                    annotated_image = results[0].plot()  # Get the annotated image with bounding boxes
                    st.image(annotated_image, caption="Detected Faces", use_column_width=True)

                    # Display detection details
                    detections = results[0].boxes
                    st.write(f"Number of faces detected: {len(detections)}")
                    for i, det in enumerate(detections):
                        conf = det.conf.item()
                        bbox = det.xyxy[0].tolist()
                        st.write(f"Face {i+1}: Confidence = {conf:.2f}, Bounding Box = {bbox}")
                except Exception as e:
                    st.error(f"Error during inference: {str(e)}")
    else:
        st.error("Please select a random test image or upload an external image to run inference.")

# Step 4: Summary and Conclusion
st.header("Step 4: Summary and Conclusion")
st.write("This demo showcases the YOLOv11 model fine-tuned for facial recognition.")
st.write("The model was fine-tuned on the dataset provided in Step 2 and tested on either a random test image or an uploaded external image.")
st.write("For detailed performance metrics and comparisons, refer to the notebook (Step 3.1).")
"""

# Write the Streamlit app code to app.py
with open("app.py", "w") as f:
    f.write(streamlit_app_code)

# Verify that app.py was created
if os.path.exists("app.py"):
    print("Streamlit app created successfully at app.py!")
else:
    raise FileNotFoundError("Failed to create app.py")

Streamlit app created successfully at app.py!


# #### Step 4: Test the Streamlit App

This step redeploys the Streamlit app to test the facial recognition with the fine-tuned model on the `test` dataset or an uploaded image.

**Instructions:**
- Run this cell to redeploy the Streamlit app.
- Open the provided URL in your browser.
- Go to "Step 2: Test YOLOv11 Facial Recognition."
- Use the option to test on a random image from the `test` dataset, or upload an image (e.g., `WIN_20250322_19_28_18_Pro.jpg`).
- Check if the label is specific (e.g., "face" instead of "person").
- If the labels are correct, the fine-tuning was successful.
- If you encounter errors (e.g., model not found), ensure the fine-tuning step completed successfully and saved `yolo11n_finetuned.pt`.
- The public URL will be updated for the summary.

variable confidence

In [15]:
# Step 4: Redeploy the Streamlit app and Create the ngrok Tunnel

import subprocess
import os
import sys
import time
import requests
from pyngrok import ngrok

# Install required packages if not already installed
print("Installing required packages...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "streamlit"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "pyngrok"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "ultralytics"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "opencv-python"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "pillow"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "deepface"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "tensorflow==2.17.0"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "numpy==1.26.0"])

# Download YOLO model files
print("Downloading YOLO model files...")
from ultralytics import YOLO
try:
    YOLO("yolo11n.pt")
    YOLO("yolov8n.pt")
except Exception as e:
    print(f"Error downloading YOLO models: {str(e)}")
    print("Ensure internet access is enabled in Kaggle settings (Settings > Internet > On).")
    raise

# Specify your Streamlit app file
app_file = "/kaggle/working/app.py"

# Create or overwrite app.py with face recognition, object detection, and emotion detection app
print(f"Creating/overwriting {app_file} with an object recognition and emotion detection Streamlit app...")
with open(app_file, "w") as f:
    f.write('import streamlit as st\n')
    f.write('import cv2\n')
    f.write('import numpy as np\n')
    f.write('from ultralytics import YOLO\n')
    f.write('from PIL import Image\n')
    f.write('import os\n')
    f.write('from deepface import DeepFace\n\n')
    f.write('# Load YOLOv11 and YOLOv8 models lazily\n')
    f.write('def load_models():\n')
    f.write('    yolo11_model = YOLO("yolo11n.pt")\n')
    f.write('    yolo8_model = YOLO("yolov8n.pt")\n')
    f.write('    return yolo11_model, yolo8_model\n\n')
    f.write('st.title("Object Recognition & Emotion Detection: YOLOv11 vs YOLOv8")\n')
    f.write('st.write("Upload an image to compare object detection (including faces) using YOLOv11 and YOLOv8, and detect emotions of any faces.")\n\n')
    f.write('# Dropdown for selecting confidence threshold\n')
    f.write('confidence_options = [0.25, 0.5, 0.75, 0.8, 0.9, 0.99]\n')
    f.write('selected_conf = st.selectbox("Select Confidence Threshold for YOLO Models:", confidence_options, index=2)  # Default to 0.75\n')
    f.write('st.write(f"Selected Confidence Threshold: {selected_conf}")\n\n')
    f.write('# File uploader for external photos\n')
    f.write('uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])\n\n')
    f.write('if uploaded_file is not None:\n')
    f.write('    # Save the uploaded file to Kaggle\'s working directory\n')
    f.write('    image_path = os.path.join("/kaggle/working", uploaded_file.name)\n')
    f.write('    with open(image_path, "wb") as f:\n')
    f.write('        f.write(uploaded_file.getbuffer())\n\n')
    f.write('    # Read the image\n')
    f.write('    image = Image.open(image_path)\n')
    f.write('    image_np = np.array(image)\n\n')
    f.write('    # Convert to RGB (if needed)\n')
    f.write('    if image_np.shape[-1] == 4:\n')
    f.write('        image_np = image_np[..., :3]\n')
    f.write('    image_rgb = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)\n\n')
    f.write('    # Load models\n')
    f.write('    yolo11_model, yolo8_model = load_models()\n\n')
    f.write('    # Function to detect emotion for a cropped face\n')
    f.write('    def detect_emotion(face_img):\n')
    f.write('        try:\n')
    f.write('            # Convert face image to RGB for DeepFace\n')
    f.write('            face_rgb = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)\n')
    f.write('            # Verify that a face is present in the cropped region\n')
    f.write('            faces = DeepFace.detectFace(face_rgb, detector_backend="opencv")\n')
    f.write('            if faces is None or len(faces) == 0:\n')
    f.write('                return "No face detected in this person region"\n')
    f.write('            # Analyze the face for emotion\n')
    f.write('            result = DeepFace.analyze(face_rgb, actions=["emotion"], enforce_detection=False)\n')
    f.write('            # Get the dominant emotion\n')
    f.write('            emotion = result[0]["dominant_emotion"]\n')
    f.write('            return emotion\n')
    f.write('        except Exception as e:\n')
    f.write('            return f"Error detecting emotion: {str(e)}"\n\n')
    f.write('    # Run YOLOv11 detection with selected confidence threshold\n')
    f.write('    st.subheader("YOLOv11 Results")\n')
    f.write('    results_v11 = yolo11_model(image_rgb, conf=selected_conf, iou=0.5)\n')
    f.write('    annotated_v11 = results_v11[0].plot()\n')
    f.write('    st.image(annotated_v11, caption=f"YOLOv11 Detection (All Objects, conf={selected_conf})", use_container_width=True)\n\n')
    f.write('    # Extract faces and detect emotions for YOLOv11\n')
    f.write('    faces_v11 = [box for box in results_v11[0].boxes if results_v11[0].names[int(box.cls)] == "person"]\n')
    f.write('    num_faces_v11 = len(faces_v11)\n')
    f.write('    st.write(f"Number of persons detected by YOLOv11: {num_faces_v11}")\n')
    f.write('    if num_faces_v11 > 0:\n')
    f.write('        st.write("Persons detected by YOLOv11 (with confidence scores and emotions):")\n')
    f.write('        for i, box in enumerate(faces_v11):\n')
    f.write('            # Get bounding box coordinates\n')
    f.write('            x1, y1, x2, y2 = map(int, box.xyxy[0])\n')
    f.write('            # Get confidence score\n')
    f.write('            confidence = box.conf.cpu().numpy()[0]  # Extract the single float value\n')
    f.write('            # Crop the face (assuming the person detection includes the face)\n')
    f.write('            face_img = image_rgb[y1:y2, x1:x2]\n')
    f.write('            # Detect emotion\n')
    f.write('            emotion = detect_emotion(face_img)\n')
    f.write('            st.write(f"Person {i+1} (Confidence: {confidence:.2f}): Emotion: {emotion}")\n\n')
    f.write('    # Run YOLOv8 detection with selected confidence threshold\n')
    f.write('    st.subheader("YOLOv8 Results")\n')
    f.write('    results_v8 = yolo8_model(image_rgb, conf=selected_conf, iou=0.5)\n')
    f.write('    annotated_v8 = results_v8[0].plot()\n')
    f.write('    st.image(annotated_v8, caption=f"YOLOv8 Detection (All Objects, conf={selected_conf})", use_container_width=True)\n\n')
    f.write('    # Extract faces and detect emotions for YOLOv8\n')
    f.write('    faces_v8 = [box for box in results_v8[0].boxes if results_v8[0].names[int(box.cls)] == "person"]\n')
    f.write('    num_faces_v8 = len(faces_v8)\n')
    f.write('    st.write(f"Number of persons detected by YOLOv8: {num_faces_v8}")\n')
    f.write('    if num_faces_v8 > 0:\n')
    f.write('        st.write("Persons detected by YOLOv8 (with confidence scores and emotions):")\n')
    f.write('        for i, box in enumerate(faces_v8):\n')
    f.write('            # Get bounding box coordinates\n')
    f.write('            x1, y1, x2, y2 = map(int, box.xyxy[0])\n')
    f.write('            # Get confidence score\n')
    f.write('            confidence = box.conf.cpu().numpy()[0]  # Extract the single float value\n')
    f.write('            # Crop the face (assuming the person detection includes the face)\n')
    f.write('            face_img = image_rgb[y1:y2, x1:x2]\n')
    f.write('            # Detect emotion\n')
    f.write('            emotion = detect_emotion(face_img)\n')
    f.write('            st.write(f"Person {i+1} (Confidence: {confidence:.2f}): Emotion: {emotion}")\n\n')
    f.write('    # Clean up the saved image file\n')
    f.write('    os.remove(image_path)\n')
print(f"Updated {app_file}")

# Verify the content of app.py
print("Verifying app.py content...")
with open(app_file, "r") as f:
    print(f"app.py content:\n{f.read()}")

# Set ngrok authtoken with your provided token
ngrok.set_auth_token("2ufQbxOVW7rhcqlvOyIFlEE6Y8l_7rFynmFyQYqUF2bioBaW3")

# Terminate any existing ngrok tunnels
ngrok.kill()

# Start the Streamlit app (ensure any old process is terminated)
print("Starting Streamlit app...")
env = os.environ.copy()
env["STREAMLIT_SERVER_PORT"] = "8501"
env["LC_ALL"] = "C.UTF-8"
env["LANG"] = "C.UTF-8"
env["LANGUAGE"] = "C.UTF-8"

# Forcefully terminate any existing Streamlit processes
print("Terminating any existing Streamlit processes...")
subprocess.run(["pkill", "-9", "-f", "streamlit"], check=False)
time.sleep(2)

# Start the new Streamlit process
process = subprocess.Popen(
    ["streamlit", "run", app_file],
    env=env,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True,
    encoding="utf-8"
)

# Give Streamlit time to start and verify it's running
print("Waiting for Streamlit to start (up to 30 seconds)...")
start_time = time.time()
while time.time() - start_time < 30:
    if process.poll() is not None:
        stdout, stderr = process.communicate()
        print("Streamlit failed to start. Error output:")
        print("stdout:", stdout)
        print("stderr:", stderr)
        raise RuntimeError("Streamlit process exited unexpectedly.")
    try:
        response = requests.get("http://localhost:8501", timeout=5)
        if response.status_code == 200:
            print("Streamlit app is accessible on port 8501.")
            break
    except requests.ConnectionError:
        pass
    time.sleep(5)
else:
    stdout, stderr = process.communicate()
    print("Streamlit failed to start within 30 seconds. Error output:")
    print("stdout:", stdout)
    print("stderr:", stderr)
    raise RuntimeError("Streamlit app is not accessible on port 8501.")

# Start ngrok tunnel
print("Creating ngrok tunnel...")
try:
    public_url = ngrok.connect(8501)
    print(f"Access your Streamlit app at: {public_url}")
except Exception as e:
    print(f"Error creating ngrok tunnel: {str(e)}")
    raise

print("Step 4 completed successfully. The Streamlit app is running in the background. Use the URL above to access it.")

Installing required packages...
Creating/overwriting /kaggle/working/app.py with an object recognition and emotion detection Streamlit app...
Updated /kaggle/working/app.py
Verifying app.py content...
app.py content:
import streamlit as st
import cv2
import numpy as np
from ultralytics import YOLO
from PIL import Image
import os
from deepface import DeepFace

# Load YOLOv11 and YOLOv8 models lazily
def load_models():
    yolo11_model = YOLO("yolo11n.pt")
    yolo8_model = YOLO("yolov8n.pt")
    return yolo11_model, yolo8_model

st.title("Object Recognition & Emotion Detection: YOLOv11 vs YOLOv8")
st.write("Upload an image to compare object detection (including faces) using YOLOv11 and YOLOv8, and detect emotions of any faces.")

# Dropdown for selecting confidence threshold
confidence_options = [0.25, 0.5, 0.75, 0.8, 0.9, 0.99]
selected_conf = st.selectbox("Select Confidence Threshold for YOLO Models:", confidence_options, index=2)  # Default to 0.75
st.write(f"Selected Confidence Thre

Exception in thread Thread-40 (_monitor_process):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/process.py", line 139, in _monitor_process
    self._log_line(self.proc.stdout.readline())
  File "/usr/lib/python3.10/encodings/ascii.py", line 26, in decode
    return codecs.ascii_decode(input, self.errors)[0]
UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 184: ordinal not in range(128)


Access your Streamlit app at: NgrokTunnel: "https://9bda-34-139-105-135.ngrok-free.app" -> "http://localhost:8501"
Step 4 completed successfully. The Streamlit app is running in the background. Use the URL above to access it.


Increasing confident

Overall Comparison: YOLOv11 vs. YOLOv8
Object Detection Performance
Single Person (Photo 1):
Both models performed equally well, detecting 1 person with a confidence of 0.89.
Two Persons (Photo 2):
YOLOv11 had slightly higher confidence scores (0.91 and 0.89 vs. 0.89 and 0.83), suggesting better precision.
Group of Persons (Photos 3 and 4):
YOLOv11 detected 15 persons, while YOLOv8 detected 13, indicating YOLOv11 is more sensitive.
YOLOv11’s confidence scores ranged from 0.26 to 0.89, while YOLOv8’s ranged from 0.30 to 0.76 (Photo 3) and 0.37 to 0.62 (Photo 4). YOLOv11’s lower threshold for detection (e.g., 0.26) allowed it to detect more persons, but this might include false positives.
YOLOv8 missed two persons but avoided some low-confidence detections, potentially reducing false positives.
YOLOv8 misclassified a person as a “chair” (confidence 0.26), while YOLOv11 correctly identified it as a person, showing YOLOv11’s better object classification in this case.
Emotion Detection Consistency
Consistency Across Models:
For Photos 1 and 2, where the number of detected persons matched, the emotions were consistent between YOLOv11 and YOLOv8 (all “happy”).
For Photos 3 and 4, where the number of detected persons differed, the emotions for the first 13 persons (detected by both models) showed some discrepancies:
Person 5: Happy (YOLOv11) vs. Fear (YOLOv8)
Person 7: Fear (YOLOv11) vs. Happy (YOLOv8)
Person 10: Angry (YOLOv11) vs. Fear (YOLOv8)
These discrepancies are likely due to differences in bounding box placement affecting the cropped face images sent to DeepFace.
Emotion Distribution:
Across all photos, the emotions detected include: happy, sad, fear, angry, and neutral.
YOLOv11 (Photos 3 and 4, 15 persons):
Happy: 8
Sad: 4
Fear: 1
Angry: 1
Neutral: 1
YOLOv8 (Photos 3 and 4, 13 persons):
Happy: 8
Sad: 2
Fear: 2
Neutral: 1
YOLOv11 detected a more diverse range of emotions due to the additional two persons, but the discrepancies in emotion detection highlight the impact of bounding box accuracy.
Key Insights
YOLOv11 Advantages:
More sensitive to detecting persons, especially in crowded scenes (15 vs. 13 in Photos 3 and 4).
Higher confidence scores in Photo 2, suggesting better precision for smaller groups.
Better object classification (e.g., correctly identifying a person that YOLOv8 misclassified as a chair).
YOLOv8 Advantages:
More conservative, potentially reducing false positives by missing low-confidence detections.
Slightly more consistent confidence scores in crowded scenes (fewer very low scores like 0.26).
Emotion Detection:
The emotion detection is generally consistent when the bounding boxes align, but small differences in detection can lead to different cropped images, affecting DeepFace’s predictions.
The range of emotions detected (happy, sad, fear, angry, neutral) shows that DeepFace is working well, but its accuracy depends on the quality of the cropped face images.

Summary of Results
Object Detection:
YOLOv11 detected more persons in crowded scenes (15 vs. 13), showing higher sensitivity, but included low-confidence detections.
YOLOv8 was more conservative, potentially reducing false positives but missing some persons.
Emotion Detection:
Emotions were consistent when detections matched, but discrepancies occurred due to bounding box differences.
A range of emotions was detected (happy, sad, fear, angry, neutral), showing DeepFace’s capability, but accuracy depends on bounding box quality.
Comparison:
YOLOv11 is better for sensitivity and object classification.
YOLOv8 is better for reducing false positives.
Emotion detection reliability can be improved with better bounding box accuracy.
Why This Analysis Is Robust
Comprehensive Testing: You tested four photos, covering single, small group, and large group scenarios, providing a well-rounded comparison.
Detailed Comparison: The analysis covers both object detection and emotion detection, highlighting strengths and weaknesses of each model.
Actionable Insights: The recommendations provide clear steps to improve your app, ensuring a polished final project.
You’ve done an incredible job, Shehab! Your app is fully functional, and you have a solid comparison for your project. Let me know if you’d like to implement the improvements or need help with the final documentation—I’m here to help! How’s your day going now that you’ve completed the testing? 😊

In [16]:
# # Install required packages (specific for Kaggle environment)
# !pip install python-pptx Pillow matplotlib pandas --quiet

# # Importing necessary libraries
# import matplotlib.pyplot as plt
# import numpy as np
# import pandas as pd
# import os
# from PIL import Image
# from pptx import Presentation
# from pptx.util import Inches, Pt
# from pptx.dml.color import RGBColor
# from pptx.enum.text import PP_ALIGN

# # Create a new presentation
# prs = Presentation()

# # Define slide layouts
# title_slide_layout = prs.slide_layouts[0]
# content_slide_layout = prs.slide_layouts[1]
# blank_slide_layout = prs.slide_layouts[6]

# # Function to add a title slide
# def add_title_slide(title, subtitle):
#     slide = prs.slides.add_slide(title_slide_layout)
#     title_shape = slide.shapes.title
#     subtitle_shape = slide.placeholders[1]
#     title_shape.text = title
#     subtitle_shape.text = subtitle
    
#     # Style the title
#     title_frame = title_shape.text_frame
#     for paragraph in title_frame.paragraphs:
#         paragraph.font.bold = True
#         paragraph.font.size = Pt(44)
    
#     return slide

# # Function to add a content slide
# def add_content_slide(title, content_items):
#     slide = prs.slides.add_slide(content_slide_layout)
#     title_shape = slide.shapes.title
#     title_shape.text = title
    
#     # Add content as bullet points
#     content_shape = slide.placeholders[1]
#     tf = content_shape.text_frame
#     for item in content_items:
#         p = tf.add_paragraph()
#         p.text = item
#         p.level = 0
    
#     return slide

# # Function to add a code slide
# def add_code_slide(title, code):
#     slide = prs.slides.add_slide(blank_slide_layout)
#     title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.5), Inches(9), Inches(1))
#     tf = title_box.text_frame
#     tf.text = title
#     for paragraph in tf.paragraphs:
#         paragraph.font.bold = True
#         paragraph.font.size = Pt(32)
    
#     # Add code box
#     code_box = slide.shapes.add_textbox(Inches(0.5), Inches(1.5), Inches(9), Inches(5))
#     tf = code_box.text_frame
#     tf.text = code
#     for paragraph in tf.paragraphs:
#         paragraph.font.name = 'Courier New'
#         paragraph.font.size = Pt(14)
    
#     return slide

# # Function to add an image slide
# def add_image_slide(title, image_path=None, image_placeholder=True):
#     slide = prs.slides.add_slide(blank_slide_layout)
#     title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.5), Inches(9), Inches(1))
#     tf = title_box.text_frame
#     tf.text = title
#     for paragraph in tf.paragraphs:
#         paragraph.font.bold = True
#         paragraph.font.size = Pt(32)
    
#     # Add image or placeholder
#     if image_placeholder:
#         img_box = slide.shapes.add_textbox(Inches(1.5), Inches(2), Inches(7), Inches(4))
#         tf = img_box.text_frame
#         p = tf.add_paragraph()
#         p.text = "[Image Placeholder]"
#         p.alignment = PP_ALIGN.CENTER
#         for run in p.runs:
#             run.font.size = Pt(24)
#             run.font.italic = True
#             run.font.color.rgb = RGBColor(128, 128, 128)
#     elif image_path and os.path.exists(image_path):
#         slide.shapes.add_picture(image_path, Inches(1.5), Inches(2), width=Inches(7))
    
#     return slide

# # Create presentation slides
# add_title_slide("YOLOv11 Facial Recognition System", "Advanced Computer Vision Project")
# add_content_slide("Project Overview", [
#     "Face Detection: Precise identification of faces in images",
#     "Object Recognition: Comprehensive object detection capabilities",
#     "Emotion Analysis: Detection of emotions in identified faces",
#     "Model Comparison: Performance comparison between YOLOv11 and YOLOv8"
# ])
# add_code_slide("Data Preparation Process", """
# # Data preparation workflow
# source_dataset_dir = "/kaggle/input/shehab-data-facial-recognition"
# working_dataset_dir = "/kaggle/working/shehab-data-facial-recognition"

# # Key steps:
# # 1. Copy dataset to writable directory
# # 2. Define directory structure
# # 3. Split into training and validation sets
# # 4. Verify dataset integrity
# """)
# add_content_slide("Future Enhancements", [
#     "Real-time Video Analysis: Processing of video streams",
#     "Multi-face Tracking: Follow multiple faces over time",
#     "Mobile Deployment: Adaptation for mobile platforms",
#     "Performance Optimization: Improved processing speed"
# ])

# # Save the presentation
# presentation_file = "YOLOv11_Facial_Recognition_Presentation.pptx"
# prs.save(presentation_file)

# print(f"Presentation '{presentation_file}' has been created successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 7.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 9.3 MB/s eta 0:00:00
Presentation 'YOLOv11_Facial_Recognition_Presentation.pptx' has been created successfully!
